#procurando palavras chave


In [1]:
#pip installs
!pip install pyspark #pyspark
!pip install nltk #nltk
!pip install dash
!pip install wordcloud

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [2]:
#imports
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import nltk
from nltk.corpus import stopwords
import requests
from bs4 import BeautifulSoup as bs
from nltk.tokenize import word_tokenize
spark = SparkSession.builder.appName("HelloWorld").getOrCreate()
sc = spark.sparkContext

In [3]:
import pandas as pd
import re
from collections import Counter
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder, TrigramAssocMeasures, TrigramCollocationFinder
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import io
import base64

In [4]:
import requests
import re
from bs4 import BeautifulSoup


In [5]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_https_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('punkt_tab') # Adicione esta linha para baixar o recurso 'punkt_tab'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
#stopwords

nltk.download('stopwords')
print(stopwords.words('portuguese'))
stopwords_pt = set(stopwords.words('portuguese'))

#adicionando mais algumas palavras no stopwords

stopwords_pt_custom = set(stopwords.words('portuguese'))
palavras_irrelevantes_adicionais = {
    'r', 'n', 'é', 'ser', 'ter', 'pode', 'https', 'http', 'www', 'com', 'br',
    'sobre', 'também', 'ainda', 'sempre', 'grande', 'tipo', 'tipos', 'forma',
    'parte', 'exemplo', 'assim', 'então', 'porque', 'pois', 'cada', 'coisa',
    'toda', 'todo', 'todos', 'todas', 'etc', 'clique', 'aqui', 'veja', 'leia',
    'artigo', 'site', 'página', 'vídeo', 'imagem', 'foto', 'conteúdo', 'post', 'mais', 'novo', 'todos', 'sempre', 'como', 'para', 'água', 'além', 'caso', 'uso'
    'janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho',
    'agosto', 'setembro', 'outubro', 'novembro', 'dezembro' # Meses, se não relevantes
}
stopwords_pt_custom.update(palavras_irrelevantes_adicionais)

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
#lista dos sites
listaSites=[]
listaSites.append("https://amominhapele.com.br/guia-cuidados-com-a-pele-diferentes-tipos")
listaSites.append("https://clinifemina.com.br/pele-sensivel")
listaSites.append("https://clinicadepele.com.br/cuidados-diarios-com-a-pele")
listaSites.append("https://drfilipefarage.com.br/melhores-praticas-de-cuidados-com-a-pele-sensivel")
listaSites.append("https://www.cetaphil.com.br/dicas-de-skincare/o-que-significa-nao-comedogenico-hipoalergenico-e-sem-perfume1663007211087.html")
listaSites.append("https://www.sbd.org.br/cuidados/cuidados-diarios-com-a-pele")
listaSites.append("https://www.bioderma.pt/sua-pele/pele-sensivel/pele-sensivel-intolerante/o-que-e-pele-sensivel-e-como-saber-se-tem-pele-sensivel")
listaSites.append("https://www.laroche-posay.pt/article/pele-alergica-pele-sensivel-e-pele-reativa")
listaSites.append("https://www.terra.com.br/vida-e-estilo/saude/alergia-a-maquiagem-como-evitar-e-tratar-o-problema,46f6350b530d649c2d18806b8f9c3a2dc8clc4qc.html")
listaSites.append("https://www.sallve.com.br/blogs/sallve/pele-irritada-como-acalmar-tratamento")
listaSites.append("https://www.mantecorpskincare.com.br/blog/skincare/dicas-do-que-fazer-quando-a-pele-esta-irritada/")
listaSites.append("https://medico24hs.com.br/blog/saude/conheca-os-tipos-de-doencas-de-pele")
listaSites.append("https://www.tuasaude.com/doencas-de-pele/")
listaSites.append("https://www.dermaclub.com.br/blog/todos-os-temas/queimaduras-de-sol.html?srsltid=AfmBOoo97zngw_V6UiRLKvS0hQ3JHUad1CEjAfaiHwooQ2XUAGcmZZFx")
listaSites.append("https://www.nebacetin.com.br/blog/cuidados-com-a-pele/o-que-e-bom-para-queimadura-de-sol-tipos-e-cuidados-especiais")
listaSites.append("https://vidasaudavel.einstein.br/picada-de-insetos-cuidados-com-a-pele-podem-evitar-lesoes")
listaSites.append("https://www.tuasaude.com/alergia-a-picada-de-inseto/")
listaSites.append("https://drauziovarella.uol.com.br/doencas-e-sintomas/ferimentos-por-agua-viva/")
listaSites.append("https://briannanicoletti.com.br/alergia-de-pele-e-calor/")
listaSites.append("https://www.biodermabrasil.com/sua-pele/pele-sensivel/pele-sensivel-intolerante/sua-pele-e-sensivel-entenda-como-lidar")
listaSites.append("https://www.terra.com.br/vida-e-estilo/saude/alergia-a-maquiagem-como-evitar-e-tratar-o-problema,46f6350b530d649c2d18806b8f9c3a2dc8clc4qc.html")

In [8]:
#toDF

def toDF(result):
    lista=[]
    for linha in result:
        dict_aux = {}
        dict_aux.update({"palavra": linha[0], "qtde": linha[1]})
        lista.append(dict_aux)
    df = pd.DataFrame(lista)
    return df

scrap tstes

In [9]:
def limpaHTML(pagina):
  soup = bs(pagina, 'html.parser')
  parag = soup.find_all('p')
  link=soup.find_all('a')
  linhas=[]
  for i in range(len(parag)):
    linhas.append(parag[i].text.upper())
  for i in range(len(link)):
    linhas.append(link[i].text.upper())
  return linhas

dados=[]
for page in listaSites:
  response = requests.get(page)
  linhas=limpaHTML(response.text)
  dados=dados+linhas

print(dados)

['CUIDAR DA PELE É ESSENCIAL PARA MANTÊ-LA SAUDÁVEL E BONITA. MAS VOCÊ SABIA QUE CADA TIPO DE PELE PRECISA DE CUIDADOS ESPECÍFICOS? VAMOS FALAR SOBRE COMO IDENTIFICAR SEU TIPO DE PELE E AS MELHORES PRÁTICAS DE CUIDADO PARA CADA UM. SEJA SUA PELE OLEOSA, SECA, MISTA OU SENSÍVEL, TEMOS DICAS E PRODUTOS PARA AJUDAR VOCÊ A MANTER SUA PELE RADIANTE.', 'ANTES DE COMEÇARMOS, É IMPORTANTE SABER QUAL É O SEU TIPO DE PELE. AQUI ESTÃO ALGUNS PASSOS SIMPLES PARA IDENTIFICAR:', 'AGORA, VEJA AS CARACTERÍSTICAS DE CADA TIPO DE PELE:', 'A PELE OLEOSA PODE SER DIFÍCIL DE GERENCIAR, MAS COM OS CUIDADOS CERTOS, VOCÊ PODE CONTROLÁ-LA.', 'A PELE SECA PRECISA DE MUITA HIDRATAÇÃO E CUIDADOS SUAVES.', 'A PELE MISTA PRECISA DE UM EQUILÍBRIO ENTRE CONTROLE DA OLEOSIDADE E HIDRATAÇÃO.', 'A PELE SENSÍVEL PRECISA DE CUIDADOS SUAVES E PRODUTOS ESPECÍFICOS PARA EVITAR IRRITAÇÕES.', 'NO AMO MINHA PELE, OFERECEMOS UMA AMPLA GAMA DE PRODUTOS PARA TODOS OS TIPOS DE PELE. AQUI ESTÃO ALGUNS DOS NOSSOS FAVORITOS:', 'AQUI E

In [10]:
import re

def func(linha_entrada, chaves): # 'chaves' parameter is unused but kept for Spark UDF compatibility
    """
    Splits the input 'linha_entrada' by commas and periods into sub-segments.
    Cleans each sub-segment and extracts all unique words longer than 3 characters from it.
    All extracted words from all sub-segments are then combined, made unique, sorted,
    and formatted into a single key string: "WORD_N,WORD_N-1,...,WORD_1,".
    """
    if not isinstance(linha_entrada, str):
        linha_entrada = str(linha_entrada) # Basic type safety

    # Split the input line by period or comma (one or more occurrences)
    # This creates sub-segments to be processed individually.
    sub_segments = re.split(r'[.,]+', linha_entrada)

    all_long_words_found = []

    for segment in sub_segments:
        # --- Text Cleaning Steps for each segment ---
        # 1. Remove HTML tags (if any) - replace with space
        cleaned_segment = re.sub(r'<[^>]+>', ' ', segment)

        # 2. Normalize newlines (\n, \r), tabs (\t), and non-breaking spaces (\xa0) to a single space
        cleaned_segment = cleaned_segment.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('\xa0', ' ')

        # 3. Replace multiple consecutive spaces with a single space
        cleaned_segment = re.sub(r'\s+', ' ', cleaned_segment)

        # 4. Trim leading/trailing whitespace from the segment
        cleaned_segment = cleaned_segment.strip()
        # --- End of Cleaning for segment ---

        if not cleaned_segment: # If segment is empty after cleaning (e.g., from "text1..text2")
            continue

        # Word extraction from the cleaned_segment
        # Regex matches sequences of letters (including common Portuguese accented ones).
        # .upper() for case normalization. Words must be > 3 letters.
        words_in_segment = re.findall(r'\b[a-zA-ZÀ-ÖØ-öø-ÿ]+\b', cleaned_segment.upper())

        for word in words_in_segment:
            if len(word) > 3: # Strictly more than 3 letters
                all_long_words_found.append(word)

    if not all_long_words_found:
        return "" # No words longer than 3 letters found in any sub-segment of the original linha_entrada

    # Make the collected words unique and sort them alphabetically
    unique_sorted_long_words = sorted(list(set(all_long_words_found)))

    # Format the output string as "WORD_N,WORD_N-1,...,WORD_1,"
    # This matches the format observed in your sample outputs.
    output_key_string = ""
    if unique_sorted_long_words:
        for word in unique_sorted_long_words:
            output_key_string = word + "," + output_key_string
            # Example: if unique_sorted_long_words is ["A", "B", "C"]
            # 1. word="A": output_key_string = "A,"
            # 2. word="B": output_key_string = "B," + "A," = "B,A,"
            # 3. word="C": output_key_string = "C," + "B,A," = "C,B,A,"

    return output_key_string

In [11]:

chaves=["ACNE", "ALÉRGICAS", "CHEIRO", "COMODOGÊNICO", "CRAVOS", "CUIDADOS",
 "DERMATOLOGISTA", "DIA", "DICAS", "FERIDAS", "FRAGRÂNCIA", "HIDRATAÇÃO",
 "HIGIENE", "HIPOALERGÊNICO", "IRRITAÇÕES", "LIMPEZA", "OLEOSA", "PELE",
 "PERFUME", "PRODUTOS", "REAÇÕES", "ROTINA", "SAÚDE", "SENSÍVEL", "SKINCARE",
 "SOLAR"]

In [12]:
input_file=sc.parallelize(dados,3)
counts = input_file.flatMap(lambda line: line.split(",")) \
                   .map(lambda linha_splitada: (func(linha_splitada, chaves), 1) if func(linha_splitada, chaves) != "" else (linha_splitada, 0)) \
                   .reduceByKey(lambda a, b: a + b)

result = counts.collect()
print(result)

# Your DataFrame conversion and filtering would then use these results.
# df = toDF(result) # Assuming toDF is a defined function to convert to a DataFrame
# dfaux = df.query("qtde > 0") # In PySpark SQL, it would be df.filter(df.qtde > 0) or df.where("qtde > 0")
# dfaux.sort_values(by="qtde", ascending=False) # In PySpark: dfaux.orderBy(dfaux.qtde.desc())

[('SENSÍVEL,MISTA,', 1), ('AGORA,', 2), ('CUIDADOS,CERTOS,', 1), ('SUAVES,SENSÍVEL,PRODUTOS,PRECISA,PELE,PARA,IRRITAÇÕES,EVITAR,ESPECÍFICOS,CUIDADOS,', 1), ('ÚTEIS,TODOS,TIPOS,PELE,PARA,GERAIS,ESTÃO,DICAS,AQUI,ALGUMAS,', 1), ('FRASCOS,', 4), ('SERUM,REVERA,FRASCOS,', 4), ('CLEAN,BEAUTY,', 4), ('FRASCO,', 2), ('', 0), ('SOBRE,PEDIDOFORMAS,PAGAMENTO,NÓSPERGUNTAS,FREQUENTESRASTREAR,', 1), ('POLITÍCAS,', 1), ('SITE,PREÇOS,PELE,PARA,PAGAMENTO,OFICIAL,NESTE,MINHA,EXCLUSIVOS,CONDIÇÕES,COMPRAS,', 1), ('PELE,', 16), ('BEENEW,', 2), ('HIALURONI,CAPS,', 2), ('HIDRAPELE,', 2), ('SMOOTHZSKIN,', 2), ('VARIZERO,', 2), ('COLÁGENO,', 2), ('MIÚRA,AMORA,', 2), ('HIDRACAPS,', 2), ('LISO,HIDRA,', 2), ('DIURIEFIT,', 2), ('DETOX,', 2), ('MOROSIL,', 2), ('LITF,DETOX,', 2), ('OZENVITTA,', 2), ('SUPLEMENTOS,', 5), ('PREMIUM,COENZIMA,', 2), ('POSTS,BLOG,', 2), ('CELULITE,', 2), ('ACESSÓRIOS,', 2), ('MAQUIAGEM,', 2), ('PELE,CUIDADO,', 1), ('PELE,PARA,NATURAIS,FUNCIONA,EVITAR,CUIDADOS,ACNEICA,', 1), ('UNIFORME,SÉR

In [13]:



# Processamento de texto para extrair palavras candidatas:
textos_concatenados_limpos = []
for linha_html in dados: # 'dados' é a sua lista de textos extraídos dos sites
    # Aplica uma limpeza similar à sua função 'func' (normalização, remoção de tags)
    # mas sem o split por '.' e ',', e mantendo o texto mais contínuo para análise.
    texto = str(linha_html)
    texto = re.sub(r'<[^>]+>', ' ', texto) # Remove HTML (se ainda houver)
    texto = texto.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('\xa0', ' ')
    texto = re.sub(r'\s+', ' ', texto).strip().lower() # Minúsculas e normaliza espaços

    if texto:
        tokens = word_tokenize(texto, language='portuguese')
        palavras_filtradas = [
            token for token in tokens
            if token.isalpha() and len(token) > 2 and token not in stopwords_pt_custom
        ]
        textos_concatenados_limpos.extend(palavras_filtradas)

# Agora 'textos_concatenados_limpos' contém uma lista de palavras candidatas
# Você pode fazer uma contagem de frequência nelas:
# from collections import Counter
# contagem_palavras_candidatas = Counter(textos_concatenados_limpos)
# print("Palavras candidatas mais frequentes:", contagem_palavras_candidatas.most_common(50))

In [14]:
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder, TrigramAssocMeasures, TrigramCollocationFinder

# Use 'textos_concatenados_limpos' da etapa anterior (já tokenizado e filtrado de stopwords)

# Bigramas
bigram_measures = BigramAssocMeasures()
finder_bigrams = BigramCollocationFinder.from_words(textos_concatenados_limpos)
finder_bigrams.apply_freq_filter(5) # Apenas bigramas que aparecem algumas vezes (ex: >5)
top_bigrams = finder_bigrams.nbest(bigram_measures.pmi, 30) # Top 30 por relevância (PMI)
print("\nBigramas Sugeridos (Termos Compostos):", top_bigrams)

# Trigramas nenhuma deu bom
trigram_measures = TrigramAssocMeasures()
finder_trigrams = TrigramCollocationFinder.from_words(textos_concatenados_limpos)
finder_trigrams.apply_freq_filter(4) # Apenas trigramas que aparecem algumas vezes (ex: >3)
top_trigrams = finder_trigrams.nbest(trigram_measures.pmi, 20)
print("\nTrigramas Sugeridos (Termos Compostos):", top_trigrams)


Bigramas Sugeridos (Termos Compostos): [('aloe', 'vera'), ('albert', 'einstein'), ('daniela', 'pimentel'), ('israelita', 'albert'), ('íntimas', 'externas'), ('zonas', 'íntimas'), ('hiperpigmentada', 'hydrabio'), ('quero', 'ler'), ('retexturizador', 'noturno'), ('fale', 'conosco'), ('científica', 'única'), ('manteiga', 'karité'), ('primeiros', 'socorros'), ('sensações', 'desagradáveis'), ('sociedade', 'brasileira'), ('filipe', 'farage'), ('dra', 'daniela'), ('rio', 'janeiro'), ('erupções', 'cutâneas'), ('envelhecimento', 'precoce'), ('abordagem', 'científica'), ('encontrar', 'ponto'), ('abordagem', 'ecobiológica'), ('olhos', 'pálpebras'), ('hospital', 'israelita'), ('couro', 'cabeludo'), ('ponto', 'venda'), ('agende', 'consulta'), ('toleriane', 'ultra'), ('disponível', 'acesso')]

Trigramas Sugeridos (Termos Compostos): [('decrypt', 'the', 'formula'), ('israelita', 'albert', 'einstein'), ('zonas', 'íntimas', 'externas'), ('revera', 'serum', 'frascos'), ('dra', 'daniela', 'pimentel'), (

#sistema automatico de pesos



In [15]:
# -*- coding: utf-8 -*-
"""
Sistema de Pesos AUTOMATIZADO para Análise de Sites de Skincare
Baseado na análise estatística dos próprios dados coletados
"""

import pandas as pd
from collections import Counter
import numpy as np
from math import log, sqrt
import re

class AutomatedSkincareSiteScorer:
    def __init__(self):
        self.word_weights = {}
        self.word_stats = {}
        self.sites_data = []

    def analyze_word_patterns(self, all_words_list, site_contents_by_url=None):
        """
        Analisa padrões das palavras nos dados para determinar pesos automaticamente

        Args:
            all_words_list: lista de todas as palavras encontradas
            site_contents_by_url: dict opcional {url: [palavras]} para análise por site
        """

        # 1. ANÁLISE DE FREQUÊNCIA GERAL
        word_freq = Counter(all_words_list)
        total_words = len(all_words_list)

        # 2. ANÁLISE DE DISTRIBUIÇÃO (TF-IDF style)
        if site_contents_by_url:
            self._calculate_site_distribution_weights(word_freq, site_contents_by_url)

        # 3. ANÁLISE DE CO-OCORRÊNCIA (palavras que aparecem juntas)
        cooccurrence_weights = self._analyze_word_cooccurrence(all_words_list)

        # 4. ANÁLISE DE POSIÇÃO/CONTEXTO (palavras em títulos, links, etc.)
        context_weights = self._analyze_word_context(all_words_list)

        # 5. ANÁLISE DE ESPECIFICIDADE (quão específicas são as palavras)
        specificity_weights = self._analyze_word_specificity(word_freq, total_words)

        # 6. COMBINA TODOS OS FATORES PARA CALCULAR PESO FINAL
        self._calculate_final_weights(word_freq, cooccurrence_weights,
                                    context_weights, specificity_weights)

        return self.word_weights

    def _calculate_site_distribution_weights(self, word_freq, site_contents_by_url):
        """Calcula pesos baseado em como as palavras se distribuem entre sites"""

        num_sites = len(site_contents_by_url)
        word_site_counts = Counter()  # quantos sites contêm cada palavra

        for url, words_in_site in site_contents_by_url.items():
            unique_words_in_site = set(words_in_site)
            for word in unique_words_in_site:
                word_site_counts[word] += 1

        # Calcula IDF (Inverse Document Frequency)
        idf_weights = {}
        for word, site_count in word_site_counts.items():
            # Palavras que aparecem em poucos sites são mais específicas
            if site_count > 0:
                idf_weights[word] = log(num_sites / site_count)
            else:
                idf_weights[word] = 0

        return idf_weights

    def _analyze_word_cooccurrence(self, all_words_list):
        """Analisa palavras que aparecem frequentemente juntas"""

        cooccurrence_scores = {}
        window_size = 5  # janela de 5 palavras para considerar co-ocorrência

        # Cria janelas deslizantes
        for i in range(len(all_words_list) - window_size + 1):
            window = all_words_list[i:i + window_size]
            unique_words_in_window = set(window)

            # Para cada par de palavras na janela, aumenta o score de co-ocorrência
            for word1 in unique_words_in_window:
                for word2 in unique_words_in_window:
                    if word1 != word2:
                        pair_key = tuple(sorted([word1, word2]))
                        cooccurrence_scores[pair_key] = cooccurrence_scores.get(pair_key, 0) + 1

        # Converte scores de pares para scores individuais
        word_cooccurrence_weights = {}
        for (word1, word2), score in cooccurrence_scores.items():
            word_cooccurrence_weights[word1] = word_cooccurrence_weights.get(word1, 0) + score
            word_cooccurrence_weights[word2] = word_cooccurrence_weights.get(word2, 0) + score

        # Normaliza os scores
        if word_cooccurrence_weights:
            max_cooccurrence = max(word_cooccurrence_weights.values())
            for word in word_cooccurrence_weights:
                word_cooccurrence_weights[word] = word_cooccurrence_weights[word] / max_cooccurrence

        return word_cooccurrence_weights

    def _analyze_word_context(self, all_words_list):
        """Analisa o contexto onde as palavras aparecem (ainda básico, pode ser expandido)"""

        context_weights = {}

        # Palavras em MAIÚSCULAS tendem a ser mais importantes
        # Palavras mais longas tendem a ser mais específicas
        # Palavras com números podem ser técnicas

        word_freq = Counter(all_words_list)

        for word, freq in word_freq.items():
            context_score = 1.0

            # Boost para palavras em maiúsculas (já que você usa .upper())
            if word.isupper() and len(word) > 3:
                context_score *= 1.2

            # Boost para palavras mais longas (mais específicas)
            if len(word) > 6:
                context_score *= 1.1
            elif len(word) > 9:
                context_score *= 1.3

            # Boost para palavras com caracteres especiais (técnicas)
            if any(char in word for char in ['Ç', 'Ã', 'Ô', 'Á', 'É']):
                context_score *= 1.1

            context_weights[word] = context_score

        return context_weights

    def _analyze_word_specificity(self, word_freq, total_words):
        """Analisa quão específicas são as palavras"""

        specificity_weights = {}

        for word, freq in word_freq.items():
            # TF (Term Frequency) normalizado
            tf = freq / total_words

            # Palavras com frequência média tendem a ser mais valiosas
            # Muito frequentes = genéricas, muito raras = podem ser ruído
            if tf > 0.01:  # Muito frequente
                specificity_score = 0.5
            elif tf < 0.0001:  # Muito rara
                specificity_score = 0.3
            else:  # Frequência ideal
                specificity_score = 1.0

            # Boost para palavras com frequência "sweet spot"
            if 0.001 <= tf <= 0.005:
                specificity_score = 1.5

            specificity_weights[word] = specificity_score

        return specificity_weights

    def _calculate_final_weights(self, word_freq, cooccurrence_weights,
                               context_weights, specificity_weights):
        """Combina todos os fatores para calcular o peso final de cada palavra"""

        for word, freq in word_freq.items():
            # Fatores individuais
            cooccurrence_factor = cooccurrence_weights.get(word, 0.1)
            context_factor = context_weights.get(word, 1.0)
            specificity_factor = specificity_weights.get(word, 1.0)

            # Fator de frequência (log para suavizar)
            frequency_factor = log(freq + 1)

            # Combina todos os fatores
            final_weight = (
                frequency_factor * 0.75 +
                cooccurrence_factor * 1.25 +
                context_factor * 1.25 +
                specificity_factor * 1.05 +
                0.5
            )

            # Normaliza para escala 1-10
            self.word_weights[word] = min(10, max(1, final_weight))

    def get_word_weight(self, word):
        """Retorna o peso calculado automaticamente para uma palavra"""
        return self.word_weights.get(word, 1.0)

    def generate_weight_report(self, top_n=50):
        """Gera relatório dos pesos calculados"""

        if not self.word_weights:
            return "Nenhum peso calculado ainda. Execute analyze_word_patterns() primeiro."

        # Ordena palavras por peso
        sorted_words = sorted(self.word_weights.items(), key=lambda x: x[1], reverse=True)

        report = f"\n=== RELATÓRIO DE PESOS AUTOMATIZADOS (TOP {top_n}) ===\n"
        report += f"{'Palavra':<20} | {'Peso':<6} | {'Categoria'}\n"
        report += "-" * 50 + "\n"

        for word, weight in sorted_words[:top_n]:
            if weight >= 7:
                category = "🔥 ALTA RELEVÂNCIA"
            elif weight >= 5:
                category = "⚡ MÉDIA RELEVÂNCIA"
            elif weight >= 3:
                category = "📌 BAIXA RELEVÂNCIA"
            else:
                category = "❌ POUCO RELEVANTE"

            report += f"{word:<20} | {weight:>5.2f} | {category}\n"

        return report

    def calculate_automated_site_score(self, word_counts, site_url=""):
        """Calcula score usando os pesos automatizados"""

        total_score = 0
        weighted_words = []
        total_words = sum(word_counts.values())

        for word, count in word_counts.items():
            weight = self.get_word_weight(word)
            word_score = count * weight
            total_score += word_score

            weighted_words.append({
                'palavra': word,
                'contagem': count,
                'peso_automatico': weight,
                'score_contribuicao': word_score
            })

        # Métricas adicionais
        avg_word_weight = total_score / total_words if total_words > 0 else 0
        high_relevance_words = len([w for w in word_counts.keys() if self.get_word_weight(w) >= 5])
        relevance_ratio = high_relevance_words / len(word_counts) if word_counts else 0

        return {
            'site_url': site_url,
            'total_score': total_score,
            'total_words': total_words,
            'avg_word_weight': avg_word_weight,
            'relevance_ratio': relevance_ratio,
            'high_relevance_word_count': high_relevance_words,
            'top_contributing_words': sorted(weighted_words, key=lambda x: x['score_contribuicao'], reverse=True)[:15]
        }

def integrar_com_codigo_existente(dados_sites, result_spark, textos_concatenados_limpos):
    """
    Integração AUTOMÁTICA com seu código existente

    Args:
        dados_sites: sua lista 'dados' com textos dos sites
        result_spark: resultado do seu processamento Spark
        textos_concatenados_limpos: sua lista de palavras limpas
    """

    print("🔄 Iniciando análise automatizada de pesos...")

    # 1. Inicializa o scorer automatizado
    auto_scorer = AutomatedSkincareSiteScorer()

    # 2. Analisa padrões automaticamente
    print("📊 Analisando padrões de palavras...")
    auto_scorer.analyze_word_patterns(textos_concatenados_limpos)

    # 3. Gera relatório de pesos
    print("📋 Gerando relatório de pesos...")
    peso_report = auto_scorer.generate_weight_report(30)
    print(peso_report)

    # 4. Converte resultado do Spark para contagem de palavras
    print("🔧 Processando resultados do Spark...")
    word_counts = {}

    for item in result_spark:
        if len(item) >= 2 and item[1] > 0:
            palavras_string = item[0]
            contagem = item[1]

            if isinstance(palavras_string, str) and palavras_string:
                palavras = [p.strip() for p in palavras_string.rstrip(',').split(',') if p.strip()]
                peso_por_palavra = contagem / len(palavras) if palavras else 0

                for palavra in palavras:
                    if palavra and len(palavra) > 2:
                        word_counts[palavra] = word_counts.get(palavra, 0) + peso_por_palavra

    # 5. Calcula score final automatizado
    print("🎯 Calculando score final automatizado...")
    score_final = auto_scorer.calculate_automated_site_score(word_counts, "Análise Automatizada")

    # 6. Relatório final
    print(f"\n🏆 === RESULTADOS FINAIS AUTOMATIZADOS ===")
    print(f"Score Total Automatizado: {score_final['total_score']:.0f}")
    print(f"Peso Médio por Palavra: {score_final['avg_word_weight']:.2f}")
    print(f"Taxa de Palavras Relevantes: {score_final['relevance_ratio']:.1%}")
    print(f"Palavras de Alta Relevância: {score_final['high_relevance_word_count']}")

    print(f"\n🔝 TOP 10 PALAVRAS QUE MAIS CONTRIBUEM (Automatizado):")
    for i, word_data in enumerate(score_final['top_contributing_words'][:10], 1):
        print(f"{i:2d}. {word_data['palavra']:15} | "
              f"Freq: {word_data['contagem']:6.1f} | "
              f"Peso: {word_data['peso_automatico']:5.2f} | "
              f"Score: {word_data['score_contribuicao']:7.1f}")

    return auto_scorer, score_final

def comparar_sites_automaticamente(lista_sites, dados_por_site):
    """
    Compara múltiplos sites usando os pesos automatizados

    Args:
        lista_sites: lista de URLs dos sites
        dados_por_site: dict {url: [palavras]} com dados de cada site
    """

    # Combina todas as palavras para análise geral
    todas_palavras = []
    for palavras_site in dados_por_site.values():
        todas_palavras.extend(palavras_site)

    # Cria scorer automatizado
    auto_scorer = AutomatedSkincareSiteScorer()
    auto_scorer.analyze_word_patterns(todas_palavras, dados_por_site)

    # Analisa cada site individualmente
    resultados_sites = []

    for url in lista_sites:
        if url in dados_por_site:
            palavras_site = dados_por_site[url]
            word_counts = Counter(palavras_site)

            score_site = auto_scorer.calculate_automated_site_score(
                dict(word_counts), url
            )
            resultados_sites.append(score_site)

    # Ordena sites por score
    resultados_sites.sort(key=lambda x: x['total_score'], reverse=True)

    print(f"\n🏆 === RANKING AUTOMATIZADO DOS SITES ===")
    for i, resultado in enumerate(resultados_sites, 1):
        print(f"{i}. {resultado['site_url']}")
        print(f"   Score: {resultado['total_score']:.0f} | "
              f"Relevância: {resultado['relevance_ratio']:.1%} | "
              f"Peso Médio: {resultado['avg_word_weight']:.2f}")
        print()

    return resultados_sites, auto_scorer

# EXEMPLO DE USO COM SEU CÓDIGO:

# Após executar todo seu código atual, adicione:

# Análise automatizada
auto_scorer, score_automatizado = integrar_com_codigo_existente(
    dados, result, textos_concatenados_limpos
)

# O sistema agora calculou automaticamente os pesos baseado nos seus dados!
# Não precisa definir pesos manualmente - tudo é baseado nos padrões encontrados.


🔄 Iniciando análise automatizada de pesos...
📊 Analisando padrões de palavras...
📋 Gerando relatório de pesos...

=== RELATÓRIO DE PESOS AUTOMATIZADOS (TOP 30) ===
Palavra              | Peso   | Categoria
--------------------------------------------------
pele                 |  8.47 | 🔥 ALTA RELEVÂNCIA
proteção             |  6.77 | ⚡ MÉDIA RELEVÂNCIA
cuidados             |  6.70 | ⚡ MÉDIA RELEVÂNCIA
solar                |  6.69 | ⚡ MÉDIA RELEVÂNCIA
tratamento           |  6.67 | ⚡ MÉDIA RELEVÂNCIA
podem                |  6.67 | ⚡ MÉDIA RELEVÂNCIA
sintomas             |  6.67 | ⚡ MÉDIA RELEVÂNCIA
produtos             |  6.57 | ⚡ MÉDIA RELEVÂNCIA
doenças              |  6.52 | ⚡ MÉDIA RELEVÂNCIA
ver                  |  6.49 | ⚡ MÉDIA RELEVÂNCIA
fazer                |  6.48 | ⚡ MÉDIA RELEVÂNCIA
sensível             |  6.45 | ⚡ MÉDIA RELEVÂNCIA
cuidado              |  6.40 | ⚡ MÉDIA RELEVÂNCIA
produto              |  6.39 | ⚡ MÉDIA RELEVÂNCIA
saúde                |  6.37 | ⚡ MÉDIA RELEV

In [16]:
auto_scorer, score_automatizado = integrar_com_codigo_existente(
    dados, result, textos_concatenados_limpos
)

🔄 Iniciando análise automatizada de pesos...
📊 Analisando padrões de palavras...
📋 Gerando relatório de pesos...

=== RELATÓRIO DE PESOS AUTOMATIZADOS (TOP 30) ===
Palavra              | Peso   | Categoria
--------------------------------------------------
pele                 |  8.47 | 🔥 ALTA RELEVÂNCIA
proteção             |  6.77 | ⚡ MÉDIA RELEVÂNCIA
cuidados             |  6.70 | ⚡ MÉDIA RELEVÂNCIA
solar                |  6.69 | ⚡ MÉDIA RELEVÂNCIA
tratamento           |  6.67 | ⚡ MÉDIA RELEVÂNCIA
podem                |  6.67 | ⚡ MÉDIA RELEVÂNCIA
sintomas             |  6.67 | ⚡ MÉDIA RELEVÂNCIA
produtos             |  6.57 | ⚡ MÉDIA RELEVÂNCIA
doenças              |  6.52 | ⚡ MÉDIA RELEVÂNCIA
ver                  |  6.49 | ⚡ MÉDIA RELEVÂNCIA
fazer                |  6.48 | ⚡ MÉDIA RELEVÂNCIA
sensível             |  6.45 | ⚡ MÉDIA RELEVÂNCIA
cuidado              |  6.40 | ⚡ MÉDIA RELEVÂNCIA
produto              |  6.39 | ⚡ MÉDIA RELEVÂNCIA
saúde                |  6.37 | ⚡ MÉDIA RELEV

#As coisas que você pediu pra deixar separadas


In [17]:
# Processamento de texto para extrair palavras candidatas:
textos_concatenados_limpos_frequencia = []
for linha_html in dados:
    texto = str(linha_html)
    texto = re.sub(r'<[^>]+>', ' ', texto)
    texto = texto.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('\xa0', ' ')
    texto = re.sub(r'\s+', ' ', texto).strip().lower()

    if texto:
        tokens = texto.split()
        palavras_filtradas = [
            token for token in tokens
            if token.isalpha() and len(token) > 2 and token not in stopwords_pt_custom
        ]
        textos_concatenados_limpos_frequencia.extend(palavras_filtradas)

from collections import Counter

contagem_palavras = Counter(textos_concatenados_limpos_frequencia)
word_freq_list = [{'Palavra': word, 'Frequência': freq} for word, freq in contagem_palavras.items()]
df_frequencia = pd.DataFrame(word_freq_list)
df_frequencia_sorted = df_frequencia.sort_values(by='Frequência', ascending=False)

print("\n--- Frequência das Palavras (Excluindo Stopwords e Palavras Curtas) ---")
print(df_frequencia_sorted.head(50))


--- Frequência das Palavras (Excluindo Stopwords e Palavras Curtas) ---
             Palavra  Frequência
1               pele         579
33          sensível          97
15          produtos          92
47             saúde          83
661         sintomas          77
432            podem          71
216   dermatologista          69
6           cuidados          62
347         proteção          61
54               ver          58
564       tratamento          57
382            solar          56
820          doenças          48
201              uso          42
160           rotina          41
1112           fazer          41
12           cuidado          41
157            ácido          39
358          produto          38
2391          picada          36
29        hidratação          34
19        importante          34
1838           gente          33
823        dermatite          33
360             deve          32
1292           corpo          32
36            evitar          32
354

In [18]:
textos_concatenados_limpos_frequencia = []
for linha_html in dados:
    texto = str(linha_html)
    texto = re.sub(r'<[^>]+>', ' ', texto)
    texto = texto.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('\xa0', ' ')
    texto = re.sub(r'\s+', ' ', texto).strip().lower()

    if texto:
        tokens = texto.split()
        palavras_filtradas = [
            token for token in tokens
            if token.isalpha() and len(token) > 2 and token not in stopwords_pt_custom
        ]
        textos_concatenados_limpos_frequencia.extend(palavras_filtradas)

contagem_palavras = Counter(textos_concatenados_limpos_frequencia)


word_freq_list = [{'Palavra': word, 'Frequência': freq} for word, freq in contagem_palavras.items()]
df_frequencia = pd.DataFrame(word_freq_list)
df_frequencia_sorted = df_frequencia.sort_values(by='Frequência', ascending=False)

print("\n--- Frequência das Palavras (Excluindo Stopwords e Palavras Curtas) ---")
print(df_frequencia_sorted.head(50).to_markdown(index=False)) # Using to_markdown for better display


collocation_tokens = []
for linha_html in dados:
    texto = str(linha_html)
    texto = re.sub(r'<[^>]+>', ' ', texto)
    texto = texto.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('\xa0', ' ')
    texto = re.sub(r'\s+', ' ', texto).strip().lower()
    if texto:
        tokens = word_tokenize(texto, language='portuguese')
        palavras_filtradas = [
            token for token in tokens
            if token.isalpha() and len(token) > 2 and token not in stopwords_pt_custom
        ]
        collocation_tokens.extend(palavras_filtradas)


# Bigramas:
bigram_measures = BigramAssocMeasures()
finder_bigrams = BigramCollocationFinder.from_words(collocation_tokens)
finder_bigrams.apply_freq_filter(5)
top_bigrams = finder_bigrams.nbest(bigram_measures.pmi, 30)

print("\n--- Bigramas Sugeridos (Termos Compostos) ---")

for i, bigram in enumerate(top_bigrams, 1):
    print(f"{i:2d}. {bigram[0]} {bigram[1]}")

# Trigramas:
trigram_measures = TrigramAssocMeasures()
finder_trigrams = TrigramCollocationFinder.from_words(collocation_tokens)
finder_trigrams.apply_freq_filter(4)
top_trigrams = finder_trigrams.nbest(trigram_measures.pmi, 20)

print("\n--- Trigramas Sugeridos (Termos Compostos) ---")

for i, trigram in enumerate(top_trigrams, 1):
    print(f"{i:2d}. {trigram[0]} {trigram[1]} {trigram[2]}")


--- Frequência das Palavras (Excluindo Stopwords e Palavras Curtas) ---
| Palavra        |   Frequência |
|:---------------|-------------:|
| pele           |          579 |
| sensível       |           97 |
| produtos       |           92 |
| saúde          |           83 |
| sintomas       |           77 |
| podem          |           71 |
| dermatologista |           69 |
| cuidados       |           62 |
| proteção       |           61 |
| ver            |           58 |
| tratamento     |           57 |
| solar          |           56 |
| doenças        |           48 |
| uso            |           42 |
| rotina         |           41 |
| fazer          |           41 |
| cuidado        |           41 |
| ácido          |           39 |
| produto        |           38 |
| picada         |           36 |
| hidratação     |           34 |
| importante     |           34 |
| gente          |           33 |
| dermatite      |           33 |
| deve           |           32 |
| corpo  

In [19]:
# 1. Contar a frequência dos bigramas sugeridos:
bigram_counts = Counter()
for bigram_tuple in top_bigrams:
    # Cria a string do bigrama para a contagem
    bigram_string = f"{bigram_tuple[0]} {bigram_tuple[1]}"

    # Percorre a lista de tokens e conta as ocorrências do bigrama como sequência
    # É mais preciso contar a sequência diretamente nos tokens
    for i in range(len(collocation_tokens) - 1):
        if collocation_tokens[i] == bigram_tuple[0] and collocation_tokens[i+1] == bigram_tuple[1]:
            bigram_counts[bigram_string] += 1

print("\n--- Frequência dos Bigramas Sugeridos ---")
# Imprime a frequência de cada bigrama sugerido
for bigram_string, count in bigram_counts.most_common():
    print(f"'{bigram_string}': {count}")

# 2. Contar a frequência dos trigramas sugeridos:
trigram_counts = Counter()
for trigram_tuple in top_trigrams:
     # Cria a string do trigrama para a contagem
    trigram_string = f"{trigram_tuple[0]} {trigram_tuple[1]} {trigram_tuple[2]}"

    # Percorre a lista de tokens e conta as ocorrências do trigrama como sequência
    for i in range(len(collocation_tokens) - 2):
        if collocation_tokens[i] == trigram_tuple[0] and collocation_tokens[i+1] == trigram_tuple[1] and collocation_tokens[i+2] == trigram_tuple[2]:
            trigram_counts[trigram_string] += 1

print("\n--- Frequência dos Trigramas Sugeridos ---")
# Imprime a frequência de cada trigrama sugerido
for trigram_string, count in trigram_counts.most_common():
     print(f"'{trigram_string}': {count}")




--- Frequência dos Bigramas Sugeridos ---
'couro cabeludo': 16
'toleriane ultra': 14
'rio janeiro': 11
'erupções cutâneas': 10
'agende consulta': 9
'disponível acesso': 9
'primeiros socorros': 8
'fale conosco': 7
'sociedade brasileira': 7
'ponto venda': 7
'albert einstein': 6
'daniela pimentel': 6
'israelita albert': 6
'íntimas externas': 6
'hiperpigmentada hydrabio': 6
'retexturizador noturno': 6
'científica única': 6
'filipe farage': 6
'dra daniela': 6
'envelhecimento precoce': 6
'abordagem científica': 6
'encontrar ponto': 6
'olhos pálpebras': 6
'hospital israelita': 6
'aloe vera': 5
'zonas íntimas': 5
'quero ler': 5
'manteiga karité': 5
'sensações desagradáveis': 5
'abordagem ecobiológica': 5

--- Frequência dos Trigramas Sugeridos ---
'sociedade brasileira dermatologia': 7
'israelita albert einstein': 6
'dra daniela pimentel': 6
'hospital israelita albert': 6
'abordagem científica única': 6
'creme retexturizador noturno': 6
'encontrar ponto venda': 6
'zonas íntimas externas': 5
'

## analise de sentimentos

In [20]:
# 1. Vocabulário da Condição Cutânea (Sintomas e Sinais Visíveis)
lexico_negativo_sintomas = {
    'vermelhidão': -0.8, # [1]
    'descamação': -0.7, # [1]
    'erupções cutâneas': -0.9, # [1]
    'inchaço': -0.7, # [1]
    'bolhas': -0.9, # [1]
    'secreção': -0.8, # [1]
    'manchas': -0.6, # [1]
    'acne': -0.8, # [1]
    'espinha': -0.7, # [1]
    'cravos': -0.6, # [1]
    'poros dilatados': -0.5, # [1]
    'rosácea': -0.7, # [1]
    'dermatite atópica': -0.8, # [1]
    'dermatite': -0.8 # [1]
}

# 2. Vocabulário da Sensação Subjetiva (Desconforto e Dor)
lexico_negativo_sensacao = {
    'coceira': -0.8, # [1]
    'coceiras': -0.8, # [1]
    'repuxamento': -0.7, # [1]
    'formigueiro': -0.6, # [1]
    'ardor': -0.8, # [1]
    'picadas': -0.7, # [1]
    'irritação': -0.9, # [1]
    'pele sensível': -0.5, # [1]
    'pele reativa': -0.6, # [1]
    'desconfortável': -0.6, # [1]
    'agredida': -0.8, # [1]
    'ressecada': -0.7, # [2, 1]
    'pele suja': -0.5 # [1]
}

# 3. Atributos e Experiências de Produto Negativas (Textura, Cheiro, etc.)
lexico_negativo_produto = {
    'pegajoso': -0.6, # [1]
    'oleoso': -0.7, # [1]
    'pesado': -0.6, # [1]
    'denso': -0.5, # [1]
    'gorduroso': -0.7, # [3, 1]
    'melequento': -0.7, # [1]
    'grudento': -0.6, # [1]
    'resíduo branco': -0.5, # [1]
    'esfarela': -0.6, # [1]
    'demora para absorver': -0.6, # [1]
    'não absorve': -0.7, # [1]
    'cheiro ruim': -0.8, # [1]
    'cheiro forte': -0.7, # [1]
    'cheiro de químico': -0.7 # [1]
}

# 4. Expressões Idiomáticas de Decepção e Falha
lexico_negativo_expressoes = {
    'piorou a irritação': -1.0, # [1]
    'me deu espinha': -0.9, # [1]
    'causou acne': -0.9, # [1]
    'obstruiu os poros': -0.8, # [1]
    'me deu alergia': -1.0, # [1]
    'não funcionou': -0.9, # [1]
    'não cumpre o que promete': -1.0, # [1]
    'não vi diferença': -0.8, # [1]
    'dinheiro jogado fora': -1.0, # [1]
    'propaganda enganosa': -1.0, # [1]
    'decepção': -0.9, # [1]
    'péssimo': -1.0, # [1]
    'horrível': -1.0, # [1]
    'um lixo': -1.0, # [1]
    'efeito rebote': -1.0, # [1]
    'deu errado': -0.9 # [4, 1]
}

In [21]:
# 1. Vocabulário dos Benefícios e Resultados Desejados
lexico_positivo_beneficios = {
    'calmante': 0.8, # [1]
    'restaurador': 0.7, # [1]
    'regenerar': 0.7, # [1]
    'anti-inflamatório': 0.8, # [1]
    'reduz a vermelhidão': 0.9, # [1]
    'alivia a coceira': 0.9, # [1]
    'acalma a pele': 0.9, # [1]
    'hidratante': 0.7, # [1]
    'hidratação profunda': 0.8, # [1]
    'restaura a barreira': 0.8, # [1]
    'pele nutrida': 0.7, # [1]
    'uniformizar o tom': 0.8, # [1]
    'pele viçosa': 0.8, # [1]
    'brilho saudável': 0.7, # [1]
    'brilho intenso': 0.7, # [5, 1]
    'pele luminosa': 0.7, # [1]
    'aparência saudável': 0.8, # [1]
    'pele bonita': 0.8, # [1]
    'pele macia': 0.6, # [1]
    'suave': 0.6, # [1]
    'toque aveludado': 0.7, # [1]
    'pele lisa': 0.7, # [1]
    'mais elástica': 0.6, # [1]
    'firme': 0.6, # [1, 6]
    'limpa suavemente': 0.8, # [1]
    'limpeza eficaz': 0.7, # [1]
    'pele limpa': 0.7, # [1]
    'protegida': 0.7 # [1]
}

# 2. Atributos e Experiências de Produto Positivas
lexico_positivo_produto = {
    'textura leve': 0.7, # [7, 1]
    'fluida': 0.6, # [7, 1]
    'aquosa': 0.6, # [7, 1]
    'refrescante': 0.7, # [7, 1]
    'não oleoso': 0.8, # [1]
    'fácil de espalhar': 0.6, # [1]
    'absorve rápido': 0.7, # [1]
    'rápida absorção': 0.7, # [1]
    'toque seco': 0.7, # [1]
    'sem cheiro': 0.8, # [1]
    'sem perfume': 0.8, # [1]
    'fragrância suave': 0.6, # [1]
    'sensação de conforto': 0.8, # [1]
    'frescor': 0.7 # [1]
}

# 3. Expressões Idiomáticas de Sucesso e Endosso
lexico_positivo_expressoes = {
    'santo graal': 1.0, # [8, 1]
    'holy grail': 1.0, # [8, 1]
    'cumpre o que promete': 1.0, # [9, 1]
    'salvou minha pele': 1.0, # [10, 1]
    'salvou meu rosto': 1.0, # [10, 1]
    'mudou minha pele': 1.0, # [1]
    'recomendo': 0.8, # [1]
    'super recomendo': 0.9, # [1]
    'compraria de novo': 0.9, # [1]
    'recomprado': 0.8, # [4, 1]
    'virou meu queridinho': 0.9, # [1]
    'não vivo sem': 1.0, # [1]
    'vale cada centavo': 0.9, # [1]
    'milagre': 1.0, # [1]
    'mágico': 0.9, # [1]
    'perfeito': 1.0, # [1]
    'incrível': 0.9, # [1]
    'maravilhoso': 0.9 # [1]
}

In [22]:
# Ingredientes 'Heróis' (Polaridade Positiva)
lexico_ingredientes_herois = {
    'niacinamida': 0.9, # [1]
    'ácido hialurônico': 0.9, # [1]
    'ceramidas': 0.9, # [1]
    'extrato de calêndula': 0.7, # [1]
    'camomila': 0.7, # [1]
    'aloe vera': 0.7, # [1]
    'prebióticos': 0.8, # [1]
    'ácido poliglutâmico': 0.8 # [1]
}

# Ingredientes 'Vilões' (Polaridade Negativa)
lexico_ingredientes_viloes = {
    'perfume': -0.9, # [1]
    'fragrância': -0.9, # [1]
    'parabenos': -0.8, # [11, 1]
    'sulfatos': -0.8, # [1]
    'corantes': -0.7, # [1]
    'álcool': -0.7 # [1]
}

In [23]:
lexico_sentimento = {}
lexico_sentimento.update(lexico_negativo_sintomas)
lexico_sentimento.update(lexico_negativo_sensacao)
lexico_sentimento.update(lexico_negativo_produto)
lexico_sentimento.update(lexico_negativo_expressoes)
lexico_sentimento.update(lexico_positivo_beneficios)
lexico_sentimento.update(lexico_positivo_produto)
lexico_sentimento.update(lexico_positivo_expressoes)
lexico_sentimento.update(lexico_ingredientes_herois)
lexico_sentimento.update(lexico_ingredientes_viloes)

In [24]:
listaSites = [
    "https://amominhapele.com.br/guia-cuidados-com-a-pele-diferentes-tipos",
    "https://clinifemina.com.br/pele-sensivel",
    "https://clinicadepele.com.br/cuidados-diarios-com-a-pele",
    "https://drfilipefarage.com.br/melhores-praticas-de-cuidados-com-a-pele-sensivel",
    "https://www.cetaphil.com.br/dicas-de-skincare/o-que-significa-nao-comedogenico-hipoalergenico-e-sem-perfume1663007211087.html",
    "https://www.sbd.org.br/cuidados/cuidados-diarios-com-a-pele",
    "https://www.bioderma.pt/sua-pele/pele-sensivel/pele-sensivel-intolerante/o-que-e-pele-sensivel-e-como-saber-se-tem-pele-sensivel",
    "https://www.laroche-posay.pt/article/pele-alergica-pele-sensivel-e-pele-reativa",
    "https://www.terra.com.br/vida-e-estilo/saude/alergia-a-maquiagem-como-evitar-e-tratar-o-problema,46f6350b530d649c2d18806b8f9c3a2dc8clc4qc.html",
    "https://www.sallve.com.br/blogs/sallve/pele-irritada-como-acalmar-tratamento",
    "https://www.mantecorpskincare.com.br/blog/skincare/dicas-do-que-fazer-quando-a-pele-esta-irritada/",
    "https://medico24hs.com.br/blog/saude/conheca-os-tipos-de-doencas-de-pele",
    "https://www.tuasaude.com/doencas-de-pele/",
    "https://www.dermaclub.com.br/blog/todos-os-temas/queimaduras-de-sol.html?srsltid=AfmBOoo97zngw_V6UiRLKv0hQ3JHUad1CEjAfaiHwooQ2XUAGcmZZFx",
    "https://www.nebacetin.com.br/blog/cuidados-com-a-pele/o-que-e-bom-para-queimadura-de-sol-tipos-e-cuidados-especiais",
    "https://vidasaudavel.einstein.br/picada-de-insetos-cuidados-com-a-pele-podem-evitar-lesoes",
    "https://www.tuasaude.com/alergia-a-picada-de-inseto/",
    "https://drauziovarella.uol.com.br/doencas-e-sintomas/ferimentos-por-agua-viva/",
    "https://briannanicoletti.com.br/alergia-de-pele-e-calor/",
    "https://www.biodermabrasil.com/sua-pele/pele-sensivel/pele-sensivel-intolerante/sua-pele-e-sensivel-entenda-como-lidar"
]


In [25]:
# --- ETAPA 2: FUNÇÃO PARA ANALISAR O SENTIMENTO DE UM ÚNICO SITE ---
# Documentação:
# Esta função recebe uma URL e o léxico. Ela faz o trabalho pesado:
# 1. Baixa o conteúdo da página web.
# 2. Usa a biblioteca BeautifulSoup para limpar o HTML e extrair apenas o texto.
# 3. Converte o texto para minúsculas e o divide em uma lista de palavras (tokens).
# 4. Itera sobre as palavras, buscando-as no léxico e somando as pontuações.
# 5. Retorna uma pontuação média para normalizar o resultado (evita que sites com mais texto tenham pontuações maiores artificialmente).

def analisar_sentimento_do_site(url, lexico):
    """
    Acessa uma URL, extrai o texto e calcula a pontuação de sentimento
    baseada em um léxico fornecido.
    """
    try:
        # Define um User-Agent para simular um navegador e evitar bloqueios
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, timeout=20, headers=headers)
        response.raise_for_status()  # Gera um erro para respostas ruins (404, 500, etc.)

        # Limpa o HTML usando BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        texto_pagina = soup.get_text()

        # Tokenização: divide o texto em palavras, remove pontuação e converte para minúsculas
        palavras = re.findall(r'\b\w+\b', texto_pagina.lower())

        # Cálculo do score de sentimento
        score_total = 0
        palavras_encontradas = 0

        for palavra in palavras:
            pontuacao = lexico.get(palavra, 0) # Retorna a pontuação da palavra ou 0 se não estiver no léxico
            if pontuacao != 0:
                score_total += pontuacao
                palavras_encontradas += 1

        # Retorna a média para normalizar a pontuação
        if palavras_encontradas > 0:
            return score_total / palavras_encontradas
        else:
            return 0.0 # Retorna 0 se nenhuma palavra do léxico for encontrada

    except requests.exceptions.RequestException as e:
        print(f"   ❌ Erro ao acessar o site {url}: {e}")
        return None # Retorna None em caso de erro de conexão

# --- ETAPA 3: EXECUÇÃO DA ANÁLISE NA LISTA DE SITES ---

# Sua lista de sites original

# Dicionário para guardar os resultados
resultados_sentimento = {}

print("\n🚀 INICIANDO ANÁLISE DE SENTIMENTOS NOS SITES...")

# Itera sobre cada site da lista
for site in listaSites:
    print(f"\n🔎 Analisando: {site}")
    score = analisar_sentimento_do_site(site, lexico_sentimento)

    if score is not None:
        resultados_sentimento[site] = score
        # Interpretação do score para uma leitura mais fácil
        interpretacao = "Neutro"
        if score > 0.1: interpretacao = "Positivo"
        elif score < -0.1: interpretacao = "Negativo"
        print(f"   📊 Score de Sentimento Médio: {score:.3f} ({interpretacao})")


# --- ETAPA 4: APRESENTAÇÃO FINAL DOS RESULTADOS ---

print("\n\n🏆 =========================================== 🏆")
print("          RESULTADO FINAL DA ANÁLISE DE SENTIMENTOS")
print("🏆 =========================================== 🏆\n")
print("Score > 0 indica um sentimento geral mais Positivo.")
print("Score < 0 indica um sentimento geral mais Negativo.")
print("Score ~ 0 indica um sentimento geral Neutro ou balanceado.\n")

# Ordena os resultados do mais positivo para o mais negativo
sites_ranqueados_por_sentimento = sorted(resultados_sentimento.items(), key=lambda item: item[1], reverse=True)

for i, (site, score) in enumerate(sites_ranqueados_por_sentimento, 1):
    print(f"{i}. Score: {score:+.3f} | Site: {site}")

print("\n✅ ANÁLISE CONCLUÍDA!")



🚀 INICIANDO ANÁLISE DE SENTIMENTOS NOS SITES...

🔎 Analisando: https://amominhapele.com.br/guia-cuidados-com-a-pele-diferentes-tipos
   📊 Score de Sentimento Médio: 0.447 (Positivo)

🔎 Analisando: https://clinifemina.com.br/pele-sensivel
   📊 Score de Sentimento Médio: -0.470 (Negativo)

🔎 Analisando: https://clinicadepele.com.br/cuidados-diarios-com-a-pele
   📊 Score de Sentimento Médio: -0.727 (Negativo)

🔎 Analisando: https://drfilipefarage.com.br/melhores-praticas-de-cuidados-com-a-pele-sensivel
   📊 Score de Sentimento Médio: -0.250 (Negativo)

🔎 Analisando: https://www.cetaphil.com.br/dicas-de-skincare/o-que-significa-nao-comedogenico-hipoalergenico-e-sem-perfume1663007211087.html
   📊 Score de Sentimento Médio: -0.407 (Negativo)

🔎 Analisando: https://www.sbd.org.br/cuidados/cuidados-diarios-com-a-pele
   📊 Score de Sentimento Médio: -0.717 (Negativo)

🔎 Analisando: https://www.bioderma.pt/sua-pele/pele-sensivel/pele-sensivel-intolerante/o-que-e-pele-sensivel-e-como-saber-se-te

In [26]:
!pip install dash_core_components
!pip install dash_html_components

In [27]:
import requests
from bs4 import BeautifulSoup
import re
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

# --- ETAPA 1: LÉXICO DE SENTIMENTOS (Exemplo) ---
# Em um cenário real, você teria um léxico maior.
# Para este exemplo, vamos usar um léxico simples.



# --- ETAPA 2: FUNÇÃO PARA ANALISAR O SENTIMENTO DE UM ÚNICO SITE ---
# Documentação:
# Esta função recebe uma URL e o léxico. Ela faz o trabalho pesado:
# 1. Baixa o conteúdo da página web.
# 2. Usa a biblioteca BeautifulSoup para limpar o HTML e extrair apenas o texto.
# 3. Converte o texto para minúsculas e o divide em uma lista de palavras (tokens).
# 4. Itera sobre as palavras, buscando-as no léxico e somando as pontuações.
# 5. Retorna uma pontuação média para normalizar o resultado (evita que sites com mais texto tenham pontuações maiores artificialmente).

def analisar_sentimento_do_site(url, lexico):
    """
    Acessa uma URL, extrai o texto e calcula a pontuação de sentimento
    baseada em um léxico fornecido.
    """
    try:
        # Define um User-Agent para simular um navegador e evitar bloqueios
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, timeout=20, headers=headers)
        response.raise_for_status()  # Gera um erro para respostas ruins (404, 500, etc.)

        # Limpa o HTML usando BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        texto_pagina = soup.get_text()

        # Tokenização: divide o texto em palavras, remove pontuação e converte para minúsculas
        palavras = re.findall(r'\b\w+\b', texto_pagina.lower())

        # Cálculo do score de sentimento
        score_total = 0
        palavras_encontradas = 0

        for palavra in palavras:
            pontuacao = lexico.get(palavra, 0) # Retorna a pontuação da palavra ou 0 se não estiver no léxico
            if pontuacao != 0:
                score_total += pontuacao
                palavras_encontradas += 1

        # Retorna a média para normalizar a pontuação
        if palavras_encontradas > 0:
            return score_total / palavras_encontradas
        else:
            return 0.0 # Retorna 0 se nenhuma palavra do léxico for encontrada

    except requests.exceptions.RequestException as e:
        print(f"   ❌ Erro ao acessar o site {url}: {e}")
        return None # Retorna None em caso de erro de conexão

# --- ETAPA 3: EXECUÇÃO DA ANÁLISE NA LISTA DE SITES ---

# Sua lista de sites original

# Dicionário para guardar os resultados
resultados_sentimento = {}

print("\n🚀 INICIANDO ANÁLISE DE SENTIMENTOS NOS SITES...")

# Itera sobre cada site da lista
for site in listaSites:
    print(f"\n🔎 Analisando: {site}")
    score = analisar_sentimento_do_site(site, lexico_sentimento)

    if score is not None:
        resultados_sentimento[site] = score
        # Interpretação do score para uma leitura mais fácil
        interpretacao = "Neutro"
        if score > 0.1: interpretacao = "Positivo"
        elif score < -0.1: interpretacao = "Negativo"
        print(f"   📊 Score de Sentimento Médio: {score:.3f} ({interpretacao})")


# --- ETAPA 4: APRESENTAÇÃO FINAL DOS RESULTADOS (CONSOLE) ---

print("\n\n🏆 =========================================== 🏆")
print("           RESULTADO FINAL DA ANÁLISE DE SENTIMENTOS")
print("🏆 =========================================== 🏆\n")
print("Score > 0 indica um sentimento geral mais Positivo.")
print("Score < 0 indica um sentimento geral mais Negativo.")
print("Score ~ 0 indica um sentimento geral Neutro ou balanceado.\n")

# Ordena os resultados do mais positivo para o mais negativo
sites_ranqueados_por_sentimento = sorted(resultados_sentimento.items(), key=lambda item: item[1], reverse=True)

for i, (site, score) in enumerate(sites_ranqueados_por_sentimento, 1):
    print(f"{i}. Score: {score:+.3f} | Site: {site}")

print("\n✅ ANÁLISE CONCLUÍDA!")


# --- ETAPA 5: CRIAÇÃO E EXECUÇÃO DO DASHBOARD ---

# Documentação:
# 1. Prepara os dados: Converte o dicionário de resultados em um DataFrame do Pandas,
#    que é um formato ideal para o Plotly.
# 2. Inicializa o App Dash: Cria a aplicação web.
# 3. Define o Layout: Estrutura a página com componentes HTML (títulos, textos) e
#    componentes Dash (o gráfico).
# 4. Cria o Gráfico: Usa Plotly Express para gerar um gráfico de barras a partir dos
#    dados. Personalizamos o gráfico com títulos e cores.
# 5. Inicia o servidor: `app.run_server` faz a mágica acontecer, rodando a aplicação
#    localmente para que você possa acessá-la pelo navegador.

if resultados_sentimento: # Apenas cria o dashboard se houver resultados
    print("\n\n📈 PREPARANDO O DASHBOARD DE VISUALIZAÇÃO...")

    # Converte o dicionário de resultados para um DataFrame do Pandas
    df_resultados = pd.DataFrame(sites_ranqueados_por_sentimento, columns=['Site', 'Score'])

    # Cria uma coluna para a cor da barra com base no score
    df_resultados['Cor'] = ['#2ca02c' if x > 0 else '#d62728' for x in df_resultados['Score']]


    # Inicializa a aplicação Dash
    app = dash.Dash(__name__)
    app.title = "Análise de Sentimentos" # Título que aparece na aba do navegador

    # Define o layout do dashboard
    app.layout = html.Div(
        style={'backgroundColor': '#f4f4f4', 'fontFamily': 'Arial, sans-serif', 'padding': '20px'},
        children=[
            html.H1(
                "Dashboard de Análise de Sentimentos de Sites",
                style={'textAlign': 'center', 'color': '#333'}
            ),
            html.P(
                "Este dashboard mostra o score de sentimento (positivo, negativo ou neutro) de cada site analisado.",
                style={'textAlign': 'center', 'color': '#666'}
            ),
            dcc.Graph(
                id='grafico-sentimentos',
                figure=px.bar(
                    df_resultados,
                    x='Site',
                    y='Score',
                    title="Score de Sentimento por Site",
                    labels={'Score': 'Score Médio de Sentimento', 'Site': 'URL do Site'},
                    text_auto='.3f' # Formata o texto na barra para 3 casas decimais
                ).update_traces(
                    marker_color=df_resultados['Cor'] # Aplica a cor personalizada
                ).update_layout(
                    xaxis_tickangle=-45, # Inclina os rótulos do eixo X para melhor leitura
                    plot_bgcolor='white', # Fundo do gráfico
                    paper_bgcolor='#f4f4f4' # Fundo da área do dashboard
                )
            )
        ]
    )

    # Executa a aplicação
    if __name__ == '__main__':
        print("\n\n🌐 Dashboard pronto! Acesse em seu navegador:")
        app.run(debug=False)

<ipython-input-27-67664244>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-27-67664244>:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html



🚀 INICIANDO ANÁLISE DE SENTIMENTOS NOS SITES...

🔎 Analisando: https://amominhapele.com.br/guia-cuidados-com-a-pele-diferentes-tipos
   📊 Score de Sentimento Médio: 0.447 (Positivo)

🔎 Analisando: https://clinifemina.com.br/pele-sensivel
   📊 Score de Sentimento Médio: -0.470 (Negativo)

🔎 Analisando: https://clinicadepele.com.br/cuidados-diarios-com-a-pele
   📊 Score de Sentimento Médio: -0.727 (Negativo)

🔎 Analisando: https://drfilipefarage.com.br/melhores-praticas-de-cuidados-com-a-pele-sensivel
   📊 Score de Sentimento Médio: -0.250 (Negativo)

🔎 Analisando: https://www.cetaphil.com.br/dicas-de-skincare/o-que-significa-nao-comedogenico-hipoalergenico-e-sem-perfume1663007211087.html
   📊 Score de Sentimento Médio: -0.407 (Negativo)

🔎 Analisando: https://www.sbd.org.br/cuidados/cuidados-diarios-com-a-pele
   📊 Score de Sentimento Médio: -0.717 (Negativo)

🔎 Analisando: https://www.bioderma.pt/sua-pele/pele-sensivel/pele-sensivel-intolerante/o-que-e-pele-sensivel-e-como-saber-se-te

<IPython.core.display.Javascript object>

#Dashboards palavras


In [28]:
def processar_textos(dados, stopwords):
    """Processa a lista de textos HTML, limpando e tokenizando as palavras."""
    tokens_limpos = []
    for linha_html in dados:
        texto = str(linha_html)
        texto = re.sub(r'<[^>]+>', ' ', texto)
        texto = texto.replace('\\n', ' ').replace('\\r', ' ').replace('\\t', ' ').replace('\\xa0', ' ')
        texto = re.sub(r'\\s+', ' ', texto).strip().lower()

        if texto:
            tokens = word_tokenize(texto, language='portuguese')
            palavras_filtradas = [
                token for token in tokens
                if token.isalpha() and len(token) > 2 and token not in stopwords
            ]
            tokens_limpos.extend(palavras_filtradas)
    return tokens_limpos

def obter_frequencia_palavras(tokens, top_n=30):
    """Retorna um DataFrame com a frequência das palavras."""
    contagem = Counter(tokens)
    df = pd.DataFrame(contagem.most_common(top_n), columns=['Palavra', 'Frequência'])
    return df

def obter_frequencia_ngrams(tokens, top_n=20):
    """Retorna DataFrames para a frequência de bigramas e trigramas."""
    # Bigramas
    bigram_measures = BigramAssocMeasures()
    finder_bigrams = BigramCollocationFinder.from_words(tokens)
    finder_bigrams.apply_freq_filter(5)
    top_bigrams = finder_bigrams.nbest(bigram_measures.pmi, top_n)
    df_bigrams = pd.DataFrame([' '.join(bigram) for bigram in top_bigrams], columns=['Bigrama'])
    # Contagem de frequência dos bigramas
    bigram_counts = Counter(zip(tokens, tokens[1:]))
    df_bigrams['Frequência'] = df_bigrams['Bigrama'].apply(lambda x: bigram_counts.get(tuple(x.split()), 0))
    df_bigrams = df_bigrams.sort_values(by='Frequência', ascending=False).head(top_n)


    # Trigramas
    trigram_measures = TrigramAssocMeasures()
    finder_trigrams = TrigramCollocationFinder.from_words(tokens)
    finder_trigrams.apply_freq_filter(4)
    top_trigrams = finder_trigrams.nbest(trigram_measures.pmi, top_n)
    df_trigrams = pd.DataFrame([' '.join(trigram) for trigram in top_trigrams], columns=['Trigrama'])
    # Contagem de frequência dos trigramas
    trigram_counts = Counter(zip(tokens, tokens[1:], tokens[2:]))
    df_trigrams['Frequência'] = df_trigrams['Trigrama'].apply(lambda x: trigram_counts.get(tuple(x.split()), 0))
    df_trigrams = df_trigrams.sort_values(by='Frequência', ascending=False).head(top_n)

    return df_bigrams, df_trigrams

# Executando a preparação dos dados
tokens_processados = processar_textos(dados, stopwords_pt_custom)
df_palavras_freq = obter_frequencia_palavras(tokens_processados)
df_bigramas_freq, df_trigramas_freq = obter_frequencia_ngrams(tokens_processados)

In [29]:
# --- Funções de Preparação de Dados (sem alterações) ---
def processar_textos(dados, stopwords):
    tokens_limpos = []
    for linha_html in dados:
        texto = str(linha_html)
        texto = re.sub(r'<[^>]+>', ' ', texto)
        texto = texto.replace('\\n', ' ').replace('\\r', ' ').replace('\\t', ' ').replace('\\xa0', ' ')
        texto = re.sub(r'\\s+', ' ', texto).strip().lower()

        if texto:
            tokens = word_tokenize(texto, language='portuguese')
            palavras_filtradas = [
                token for token in tokens
                if token.isalpha() and len(token) > 2 and token not in stopwords
            ]
            tokens_limpos.extend(palavras_filtradas)
    return tokens_limpos

def obter_frequencia_palavras(tokens, top_n=30):
    contagem = Counter(tokens)
    return pd.DataFrame(contagem.most_common(top_n), columns=['Palavra', 'Frequência'])

def obter_frequencia_ngrams(tokens, top_n=20):
    bigram_measures = BigramAssocMeasures()
    finder_bigrams = BigramCollocationFinder.from_words(tokens)
    finder_bigrams.apply_freq_filter(5)
    top_bigrams = finder_bigrams.nbest(bigram_measures.pmi, top_n)
    df_bigrams = pd.DataFrame([' '.join(bigram) for bigram in top_bigrams], columns=['Bigrama'])
    bigram_counts = Counter(zip(tokens, tokens[1:]))
    df_bigrams['Frequência'] = df_bigrams['Bigrama'].apply(lambda x: bigram_counts.get(tuple(x.split()), 0))
    df_bigrams = df_bigrams.sort_values(by='Frequência', ascending=False).head(top_n)

    trigram_measures = TrigramAssocMeasures()
    finder_trigrams = TrigramCollocationFinder.from_words(tokens)
    finder_trigrams.apply_freq_filter(4)
    top_trigrams = finder_trigrams.nbest(trigram_measures.pmi, top_n)
    df_trigrams = pd.DataFrame([' '.join(trigram) for trigram in top_trigrams], columns=['Trigrama'])
    trigram_counts = Counter(zip(tokens, tokens[1:], tokens[2:]))
    df_trigrams['Frequência'] = df_trigrams['Trigrama'].apply(lambda x: trigram_counts.get(tuple(x.split()), 0))
    df_trigrams = df_trigrams.sort_values(by='Frequência', ascending=False).head(top_n)

    return df_bigrams, df_trigramas_freq

# --- NOVA FUNÇÃO: Gerador da Nuvem de Palavras ---
def criar_nuvem_de_palavras(tokens):
    """
    Gera uma imagem de nuvem de palavras e a codifica para exibição no Dash.
    """
    # Junta todos os tokens em um único texto
    texto_completo = ' '.join(tokens)

    # Cria o objeto WordCloud
    wc = WordCloud(
        background_color="white",
        width=800,
        height=500,
        max_words=100, # Limita o número de palavras na nuvem
        contour_width=3,
        contour_color='steelblue'
    )

    # Gera a nuvem a partir do texto
    wc.generate(texto_completo)

    # Converte a imagem para um formato que o Dash pode exibir (Base64)
    buffer = io.BytesIO()
    wc.to_image().save(buffer, format="PNG")
    img_str = base64.b64encode(buffer.getvalue()).decode()

    return f"data:image/png;base64,{img_str}"


# --- Executando a preparação dos dados ---
tokens_processados = processar_textos(dados, stopwords_pt_custom)
df_palavras_freq = obter_frequencia_palavras(tokens_processados)
df_bigramas_freq, df_trigramas_freq = obter_frequencia_ngrams(tokens_processados)
imagem_nuvem_base64 = criar_nuvem_de_palavras(tokens_processados)

# --- Construção do Dashboard (com a nuvem adicionada) ---
app_palavras = dash.Dash(__name__)

app_palavras.layout = html.Div(style={'backgroundColor': '#f0f0f0', 'fontFamily': 'Arial'}, children=[
    html.H1(
        "Dashboard de Análise de Frequência de Palavras",
        style={'textAlign': 'center', 'color': '#333', 'padding': '20px'}
    ),

    # --- Seção da Nuvem de Palavras ---
    html.Div([
        html.H2("Nuvem de Palavras", style={'textAlign': 'center'}),
        html.Img(src=imagem_nuvem_base64, style={'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'})
    ], style={'padding': '20px', 'border': '1px solid #ddd', 'borderRadius': '5px', 'margin': '10px', 'backgroundColor': 'white'}),

    html.Hr(),

    # Seção dos Gráficos de Barras
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(className='six columns', style={'flex': '50%', 'padding': '10px'}, children=[
            dcc.Graph(
                id='grafico-frequencia-palavras',
                figure=px.bar(
                    df_palavras_freq.sort_values('Frequência', ascending=True),
                    x='Frequência', y='Palavra', orientation='h', title='Top 30 Palavras Frequentes',
                    template='plotly_white'
                ).update_layout(title_x=0.5)
            )
        ]),
        html.Div(className='six columns', style={'flex': '50%', 'padding': '10px'}, children=[
            dcc.Graph(
                id='grafico-frequencia-bigramas',
                figure=px.bar(
                    df_bigramas_freq.sort_values('Frequência', ascending=True),
                    x='Frequência', y='Bigrama', orientation='h', title='Top 20 Bigramas Frequentes',
                    template='plotly_white'
                ).update_layout(title_x=0.5, margin=dict(l=150))
            )
        ])
    ])
])

# Para rodar no Google Colab/Jupyter
if __name__ == '__main__':
    app_palavras.run(mode='inline')


<IPython.core.display.Javascript object>

#Rank de sites

In [30]:
# ==============================================================================
#  1. IMPORTAÇÕES E CONFIGURAÇÕES GERAIS
# ==============================================================================
import math
import re
import requests
from collections import Counter
from bs4 import BeautifulSoup as bs
import itertools

# ==============================================================================
#  ⚙️ PAINEL DE CONTROLE - AJUSTE SEUS PARÂMETROS AQUI ⚙️
# ==============================================================================

PALAVRAS_SEMENTE = {
    'pele', 'rosto', 'skincare', 'dermatologista', 'acne', 'hidratante',
    'protetor', 'sensível', 'oleosidade', 'rugas', 'manchas', 'colágeno',
    'vitamina', 'sérum', 'limpeza', 'espinhas', 'cravos', 'hipoalergênico',
    'comedogênico', 'rejuvenescedor', 'antissinais', 'clareador', 'ácido',
    'facial', 'corporal', 'cutânea', 'derme', 'epiderme', 'cutânea', 'alergia', 'alérgica',
    'maquiagem', 'alergico', 'antialergico', 'limpo', 'reações', 'reação', 'vermelhidão',
    'coceira', 'inchaço', 'bolhas', 'dermatologista', 'aliviar', 'alivio', 'anti-histamínicos',
    'hipoalergênicos', 'hipoalergênica', 'hipoalergênicas', 'reação'
}

lista2 = [
    'pele', 'sintomas', 'cuidados', 'solar', 'tratamento', 'doenças', 'proteção',
    'sensível', 'irritada', 'cuidado', 'produtos', 'rotina', 'hidratação',
    'dermatologista', 'vermelhidão', 'saúde', 'dermatite', 'rosto', 'ácido',
    'queimadura', 'limpeza', 'produto', 'picada', 'skincare', 'acne',
    'hidratante', 'protetor', 'oleosidade', 'rugas', 'manchas', 'colágeno',
    'vitamina', 'sérum', 'espinhas', 'cravos', 'hipoalergênico', 'comedogênico',
    'rejuvenescedor', 'antissinais', 'clareador', 'facial', 'corporal', 'cutânea',
    'derme', 'epiderme', 'alergia', 'alérgica', 'maquiagem', 'alergico',
    'antialergico', 'limpo', 'reações', 'reação', 'coceira', 'inchaço', 'bolhas',
    'aliviar', 'alivio', 'anti-histamínicos', 'hipoalergênicos', 'hipoalergênica',
    'hipoalergênicas'
]

for n in lista2:
  if n not in PALAVRAS_SEMENTE:
    PALAVRAS_SEMENTE.add(n)

PESO_TEMA = 0.5
PESO_RELEVANCIA = 0.2
PESO_SCORE = 0.15
PESO_MEDIO = 0.15

# --- MUDANÇA 1: Adicionando o Fator de Amplificação ---
# Um valor menor que 1 "puxa" as notas para cima.
# 0.7 é um bom começo. Valores como 0.6 ou 0.5 darão um boost ainda maior.
FATOR_AMPLIFICACAO = 0.5

# ==============================================================================
#  2. FUNÇÕES DE CÁLCULO E NORMALIZAÇÃO
# ==============================================================================
def funcao_ativacao_relevancia_v2(x):
    a = 25.5
    b = 0.123
    if x <= 0: return 0.0
    return 1.0 / (1.0 + math.exp(-a * (x - b)))

def normalizar(valor, minimo, maximo):
    faixa = maximo - minimo
    if faixa == 0: return 0.5
    resultado_normalizado = (valor - minimo) / faixa
    return max(0.0, min(1.0, resultado_normalizado))

# --- MUDANÇA 2: Aplicando a amplificação na nota final ---
def calcular_nota_final(score_bruto, peso_medio_bruto, relevancia_contextual, score_tematico, **kwargs):
    relevancia_normalizada = funcao_ativacao_relevancia_v2(relevancia_contextual)
    score_normalizado = normalizar(score_bruto, kwargs['min_score'], kwargs['max_score'])
    peso_medio_normalizado = normalizar(peso_medio_bruto, kwargs['min_peso'], kwargs['max_peso'])
    score_tematico_normalizado = normalizar(score_tematico, kwargs['min_tema'], kwargs['max_tema'])

    nota_intermediaria = (score_tematico_normalizado * PESO_TEMA) + \
                         (relevancia_normalizada * PESO_RELEVANCIA) + \
                         (score_normalizado * PESO_SCORE) + \
                         (peso_medio_normalizado * PESO_MEDIO)

    # Aplicando a curva de amplificação para "esticar" as notas
    nota_amplificada = nota_intermediaria ** FATOR_AMPLIFICACAO

    return nota_amplificada * 10

# ==============================================================================
#  3. FUNÇÕES E CLASSES DE ANÁLISE DE CONTEÚDO
# ==============================================================================
def limpaHTML(pagina):
  soup = bs(pagina, 'html.parser')
  tags = soup.find_all(['p', 'a', 'h1', 'h2', 'h3', 'li', 'span', 'strong'])
  return [tag.text for tag in tags]

def limpar_e_obter_palavras(texto, stopwords_pt_custom):
    if not isinstance(texto, str): texto = str(texto)
    cleaned_text = re.sub(r'<[^>]+>', ' ', texto)
    cleaned_text = cleaned_text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('\xa0', ' ')
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip().lower()
    if not cleaned_text: return []
    tokens = cleaned_text.split()
    return [token for token in tokens if token.isalpha() and len(token) > 2 and token not in stopwords_pt_custom]

class AdvancedSiteScorer:
    def __init__(self, palavras_semente):
        self.word_weights = {}
        self.palavras_semente = palavras_semente

    def analyze_word_patterns(self, all_words_list, site_contents_by_url=None):
        word_freq = Counter(all_words_list)
        idf_weights = self._calculate_idf_weights(site_contents_by_url)
        for word, freq in word_freq.items():
            idf_factor = idf_weights.get(word, 0)
            freq_bonus = math.log1p(freq) / 10.0
            final_weight = 1.0 + (idf_factor * 2.0) + freq_bonus
            if word in self.palavras_semente:
                final_weight *= 1.5
            self.word_weights[word] = min(10, max(1, final_weight))

    def _calculate_idf_weights(self, site_contents_by_url):
        if not site_contents_by_url: return {}
        num_sites = len(site_contents_by_url)
        if num_sites == 0: return {}
        word_site_counts = Counter()
        for words_in_site in site_contents_by_url.values():
            for word in set(words_in_site):
                word_site_counts[word] += 1
        idf_weights = {}
        for word, site_count in word_site_counts.items():
            idf_weights[word] = math.log(1 + (num_sites / site_count))
        return idf_weights

    def get_word_weight(self, word):
        return self.word_weights.get(word, 1.0)

    def calculate_site_score(self, word_counts, site_url=""):
        total_score, total_words_in_site = 0, sum(word_counts.values())
        if total_words_in_site == 0:
            return {'site_url': site_url, 'total_score': 0, 'avg_word_weight': 0, 'relevance_ratio': 0, 'relevancia_contextual': 0, 'score_tematico': 0, 'palavras_nucleares_encontradas': set(), 'palavras_relevantes_encontradas': [], 'palavras_maior_contribuicao': []}

        weighted_word_details = []
        for word, count in word_counts.items():
            weight = self.get_word_weight(word)
            word_score = count * weight
            total_score += word_score
            weighted_word_details.append({'palavra': word, 'peso': weight, 'score': word_score})

        avg_word_weight = total_score / total_words_in_site

        words_with_high_weight = [w for w in word_counts.keys() if self.get_word_weight(w) >= 5.0]
        relevance_ratio = len(words_with_high_weight) / len(word_counts) if word_counts else 0

        found_nuclear_words = {word for word in word_counts.keys() if word in self.palavras_semente}
        contagem_palavras_nucleares = sum(word_counts.get(p, 0) for p in found_nuclear_words)
        score_tematico = contagem_palavras_nucleares / total_words_in_site

        relevancia_contextual = relevance_ratio * score_tematico

        return {
            'site_url': site_url,
            'total_score': total_score,
            'avg_word_weight': avg_word_weight,
            'relevance_ratio': relevance_ratio,
            'score_tematico': score_tematico,
            'relevancia_contextual': relevancia_contextual,
            'palavras_nucleares_encontradas': found_nuclear_words,
            'palavras_relevantes_encontradas': words_with_high_weight,
            'palavras_maior_contribuicao': sorted(weighted_word_details, key=lambda x: x['score'], reverse=True)
        }

# ==============================================================================
#  4. PAINEL DE DIAGNÓSTICO
# ==============================================================================
def gerar_relatorio_detalhado(resultado_site, scorer):
    # (Esta função permanece igual)
    print("-" * 60)
    print(f"🔎 PAINEL DE DIAGNÓSTICO PARA: {resultado_site['site_url']}")
    print("-" * 60)
    print(f"⭐ Nota Final: {resultado_site.get('nota_final', 0):.2f} / 10\n")
    print(f"📊 Métricas Principais:")
    print(f"  - Score Temático (densidade de palavras-semente): {resultado_site['score_tematico']:.2%}")
    print(f"  - Relevância Bruta (palavras raras com peso > 5): {resultado_site['relevance_ratio']:.2%}")
    print(f"  - ⭐ Relevância Contextual (Relevância * Tema): {resultado_site['relevancia_contextual']:.4f}")
    print(f"  - Score Bruto (soma de contagem * peso): {resultado_site['total_score']:.0f}")
    print(f"  - Peso Médio por Palavra: {resultado_site['avg_word_weight']:.2f}")

    print("\n🧠 Análise de Tema:")
    nucleares_encontradas = resultado_site.get('palavras_nucleares_encontradas')
    if nucleares_encontradas:
        print(f"  - {len(nucleares_encontradas)} Palavras-Semente encontradas: {', '.join(sorted(list(nucleares_encontradas)))}")
    else:
        print("  - Nenhuma Palavra-Semente encontrada.")

    print("\n⚖️ Análise de Relevância (Top 10 Palavras com Peso > 5.0):")
    relevantes_encontradas = resultado_site.get('palavras_relevantes_encontradas', [])
    if relevantes_encontradas:
        pesos = {p: scorer.get_word_weight(p) for p in relevantes_encontradas}
        top_10_relevantes = sorted(pesos.items(), key=lambda x: x[1], reverse=True)[:10]
        for palavra, peso in top_10_relevantes:
            print(f"  - '{palavra}' (Peso: {peso:.2f})")
    else:
        print("  - Nenhuma palavra com peso alto encontrada.")

    print("\n📈 Análise de Score (Top 10 Palavras de Maior Contribuição):")
    maior_contribuicao = resultado_site.get('palavras_maior_contribuicao', [])
    if maior_contribuicao:
        for item in maior_contribuicao[:10]:
            print(f"  - '{item['palavra']}' (Score: {item['score']:.0f})")
    else:
        print("  - Nenhuma contribuição para o score.")
    print("-" * 60 + "\n")

# ==============================================================================
#  🚀 FUNÇÃO PRINCIPAL DE ANÁLISE E RANKING 🚀
# ==============================================================================
def analise_completa_sites(lista_sites, stopwords_pt_custom, palavras_semente):
    dados_por_site, todas_palavras_combinadas = {}, []
    print("--- ETAPA 1: Buscando e processando dados dos sites ---")
    for url in lista_sites:
        try:
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
            response = requests.get(url, timeout=15, headers=headers)
            response.raise_for_status()
            linhas_html = limpaHTML(response.text)
            palavras_do_site = [p for linha in linhas_html for p in limpar_e_obter_palavras(linha, stopwords_pt_custom)]
            if palavras_do_site:
                dados_por_site[url] = palavras_do_site
                todas_palavras_combinadas.extend(palavras_do_site)
                print(f"✅ Processado: {url}")
        except requests.exceptions.RequestException as e: print(f"❌ Erro ao buscar {url}: {e}")

    print("\n--- ETAPA 2: Análise de conteúdo e cálculo de pesos ---")
    keyword_finder = AdvancedSiteScorer(palavras_semente)
    keyword_finder.analyze_word_patterns(todas_palavras_combinadas, dados_por_site)
    resultados_brutos = [keyword_finder.calculate_site_score(Counter(p), url) for url, p in dados_por_site.items() if p]

    if not resultados_brutos: return [], None

    LIMIAR_TEMATICO = 0.02

    print("\n--- ETAPA 3: Calibrando e calculando a Nota Final ---")
    resultados_validos = [r for r in resultados_brutos if r['score_tematico'] >= LIMIAR_TEMATICO]

    if not resultados_validos:
        print("\n⚠️ Nenhum site atingiu o limiar temático mínimo. Nenhum resultado para ranquear.")
        for r in resultados_brutos:
            r['nota_final'] = 0
        sites_ranqueados = sorted(resultados_brutos, key=lambda r: r['site_url'])
        return sites_ranqueados, keyword_finder

    kwargs_dinamicos = {
        'min_score': min(r['total_score'] for r in resultados_validos), 'max_score': max(r['total_score'] for r in resultados_validos),
        'min_peso': min(r['avg_word_weight'] for r in resultados_validos), 'max_peso': max(r['avg_word_weight'] for r in resultados_validos),
        'min_tema': min(r['score_tematico'] for r in resultados_validos), 'max_tema': max(r['score_tematico'] for r in resultados_validos)
    }

    for r in resultados_brutos:
        if r['score_tematico'] < LIMIAR_TEMATICO:
            r['nota_final'] = 0.0
        else:
            r['nota_final'] = calcular_nota_final(r['total_score'], r['avg_word_weight'], r['relevancia_contextual'], r['score_tematico'], **kwargs_dinamicos)

    sites_ranqueados = sorted(resultados_brutos, key=lambda r: r.get('nota_final', 0), reverse=True)

    print(f"\n\n🏆 =========================================== 🏆")
    print(f"       RANKING FINAL DOS SITES (NOTA 0 A 10)")
    print(f"🏆 =========================================== 🏆\n")
    for i, r in enumerate(sites_ranqueados, 1):
        print(f"{i}. {r['site_url']}")
        print(f"   └───> ⭐ Nota Final: {r['nota_final']:.2f} / 10")

    print("\n\n🕵️‍♂️ =========================================== 🕵️‍♂️")
    print(f"       PAINEL DE DIAGNÓSTICO (TOP {z} SITES)")
    print(f"🕵️‍♂️ =========================================== 🕵️‍♂️")
    for r in sites_ranqueados[:z]:
        gerar_relatorio_detalhado(r, keyword_finder)

    return sites_ranqueados, keyword_finder

# ==============================================================================
#  ❓ EXECUÇÃO DO SCRIPT ❓
# ==============================================================================
z = 3

if __name__ == '__main__':
    # (A lista de stopwords e a lista de sites permanecem as mesmas)
    stopwords_pt_custom = set(['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvéssemos', 'isso', 'isto', 'já', 'lhe', 'lhes', 'mais', 'mas', 'me', 'mesmo', 'meu', 'meus', 'minha', 'minhas', 'muito', 'na', 'não', 'nas', 'nem', 'no', 'nos', 'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'num', 'numa', 'o', 'os', 'ou', 'para', 'pela', 'pelas', 'pelo', 'pelos', 'por', 'qual', 'quando', 'que', 'quem', 'são', 'se', 'seja', 'sejam', 'sejamos', 'sem', 'ser', 'será', 'serão', 'serei', 'seremos', 'seria', 'seriam', 'seríamos', 'seu', 'seus', 'só', 'somos', 'sou', 'sua', 'suas', 'também', 'te', 'tem', 'tém', 'temos', 'tenha', 'tenham', 'tenhamos', 'tenho', 'terá', 'terão', 'terei', 'teremos', 'teria', 'teriam', 'teríamos', 'teu', 'teus', 'teve', 'tinha', 'tinham', 'tínhamos', 'tive', 'tivemos', 'tiver', 'tivera', 'tiveram', 'tivéramos', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'tu', 'tua', 'tuas', 'um', 'uma', 'você', 'vocês', 'vos'])
    palavras_irrelevantes_adicionais = {'r', 'n', 'é', 'ser', 'ter', 'pode', 'https', 'http', 'www', 'com', 'br', 'sobre', 'também', 'ainda', 'sempre', 'grande', 'tipo', 'tipos', 'forma', 'parte', 'exemplo', 'assim', 'então', 'porque', 'pois', 'cada', 'coisa', 'toda', 'todo', 'todos', 'todas', 'etc', 'clique', 'aqui', 'veja', 'leia', 'artigo', 'site', 'página', 'vídeo', 'imagem', 'foto', 'conteúdo', 'post', 'mais', 'novo', 'todos', 'sempre', 'como', 'para', 'água', 'além', 'caso', 'uso', 'janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro'}
    stopwords_pt_custom.update(palavras_irrelevantes_adicionais)
    listaSites2 = [ 'https://pt.wikipedia.org/wiki/Corpo_humano','https://pt.wikipedia.org/wiki/Pele','https://pt.wikipedia.org/wiki/Reiquiavique', 'https://www.leagueoflegends.com/pt-br/', 'https://pt.m.wikipedia.org/wiki/Automóvel','https://g1.globo.com/meio-ambiente/noticia/2025/05/11/cachorros-e-gatos-estao-evoluindo-para-ficarem-parecidos-entre-si-e-isso-esta-sendo-feito-pelos-humanos.ghtml', "https://amominhapele.com.br/guia-cuidados-com-a-pele-diferentes-tipos", "https://drfilipefarage.com.br/melhores-praticas-de-cuidados-com-a-pele-sensivel", "https://www.sbd.org.br/cuidados/cuidados-diarios-com-a-pele", "https://www.biodermabrasil.com/sua-pele/pele-sensivel/pele-sensivel-intolerante/sua-pele-e-sensivel-entenda-como-lidar", "https://www.bioderma.pt/sua-pele/pele-sensivel/pele-sensivel-intolerante/o-que-e-pele-sensivel-e-como-saber-se-tem-pele-sensivel", "https://www.laroche-posay.pt/article/pele-alergica-pele-sensivel-e-pele-reativa", "https://www.terra.com.br/vida-e-estilo/saude/alergia-a-maquiagem-como-evitar-e-tratar-o-problema,46f6350b530d649c2d18806b8f9c3a2dc8clc4qc.html", "https://www.sallve.com.br/blogs/sallve/pele-irritada-como-acalmar-tratamento", "https://www.mantecorpskincare.com.br/blog/skincare/dicas-do-que-fazer-quando-a-pele-esta-irritada/", "https://medico24hs.com.br/blog/saude/conheca-os-tipos-de-doencas-de-pele", "https://www.tuasaude.com/doencas-de-pele/", "https://www.nebacetin.com.br/blog/cuidados-com-a-pele/o-que-e-bom-para-queimadura-de-sol-tipos-e-cuidados-especiais", "https://www.tuasaude.com/alergia-a-picada-de-inseto/", "https://drauziovarella.uol.com.br/doencas-e-sintomas/ferimentos-por-agua-viva/" ]
    for n in listaSites2:
      if n not in listaSites: listaSites.append(n)
    print("🚀 INICIANDO ANÁLISE COMPLETA DOS SITES...")
    resultados_finais, keyword_finder = analise_completa_sites(
        listaSites,
        stopwords_pt_custom,
        PALAVRAS_SEMENTE
    )
    print("\n✅ ANÁLISE CONCLUÍDA!")


🚀 INICIANDO ANÁLISE COMPLETA DOS SITES...
--- ETAPA 1: Buscando e processando dados dos sites ---
✅ Processado: https://amominhapele.com.br/guia-cuidados-com-a-pele-diferentes-tipos
✅ Processado: https://clinifemina.com.br/pele-sensivel
✅ Processado: https://clinicadepele.com.br/cuidados-diarios-com-a-pele
✅ Processado: https://drfilipefarage.com.br/melhores-praticas-de-cuidados-com-a-pele-sensivel
✅ Processado: https://www.cetaphil.com.br/dicas-de-skincare/o-que-significa-nao-comedogenico-hipoalergenico-e-sem-perfume1663007211087.html
✅ Processado: https://www.sbd.org.br/cuidados/cuidados-diarios-com-a-pele
✅ Processado: https://www.bioderma.pt/sua-pele/pele-sensivel/pele-sensivel-intolerante/o-que-e-pele-sensivel-e-como-saber-se-tem-pele-sensivel
✅ Processado: https://www.laroche-posay.pt/article/pele-alergica-pele-sensivel-e-pele-reativa
✅ Processado: https://www.terra.com.br/vida-e-estilo/saude/alergia-a-maquiagem-como-evitar-e-tratar-o-problema,46f6350b530d649c2d18806b8f9c3a2dc8cl

#dashboards sites

In [31]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc, html

# (Certifique-se de que a sua variável 'resultados_finais' da análise principal já foi criada)
# (Certifique-se de que as funções como 'funcao_ativacao_relevancia_v2', 'normalizar' e os PESOS estão definidos)


def encurtar_url(url):
    """
    Converte uma URL completa em um nome curto e legível para o dashboard.
    """
    # Dicionário para nomes personalizados, para um resultado mais elegante
    nomes_personalizados = {
        'sbd': 'SBD',
        'drauziovarella': 'Drauzio Varella',
        'mantecorpskincare': 'Mantecorp',
        'amominhapele': 'Amo Minha Pele',
        'drfilipefarage': 'Dr. Filipe Farage',
        'tuasaude': 'Tua Saúde',
        'nebacetin': 'Nebacetin',
        'clinifemina': 'Clinifemina',
        'cetaphil': 'Cetaphil',
        'bioderma': 'Bioderma (PT)',
        'laroche-posay': 'La Roche-Posay (PT)',
        'sallve': 'Sallve',
        'medico24hs': 'Médico 24hs',
        'biodermabrasil': 'Bioderma (BR)',
        'terra': 'Terra',
        'leagueoflegends': 'LoL (Outlier)',
        'g1': 'G1 (Outlier)'
        # Adicione outros mapeamentos personalizados se necessário
    }

    try:
        # Limpeza inicial da URL
        nome_limpo = url.replace('https://www.', '').replace('http://www.', '').replace('https://', '').replace('http://', '')
        # Pega a parte principal do domínio (ex: 'amominhapele' de 'amominhapele.com.br/...')
        partes_dominio = nome_limpo.split('/')[0].split('.')
        parte_principal = partes_dominio[0] if len(partes_dominio) > 1 else partes_dominio[0]

        # Retorna o nome personalizado se existir, senão o nome tratado
        return nomes_personalizados.get(parte_principal, parte_principal.capitalize())
    except:
        return url # Retorna a URL original em caso de erro


def obter_dados_para_dashboard_scores(resultados_finais):
    """Prepara os dados para o dashboard de scores, já com os nomes curtos."""
    dados_grafico = []

    resultados_validos = [r for r in resultados_finais if r.get('nota_final', 0) > 0]
    if not resultados_validos: return pd.DataFrame()

    kwargs_dinamicos = {
        'min_score': min(r['total_score'] for r in resultados_validos), 'max_score': max(r['total_score'] for r in resultados_validos),
        'min_peso': min(r['avg_word_weight'] for r in resultados_validos), 'max_peso': max(r['avg_word_weight'] for r in resultados_validos),
        'min_tema': min(r['score_tematico'] for r in resultados_validos), 'max_tema': max(r['score_tematico'] for r in resultados_validos)
    }

    for res in resultados_finais:
        # APLICANDO A NOVA FUNÇÃO AQUI
        site = encurtar_url(res['site_url'])
        nota_final = res.get('nota_final', 0)

        if nota_final > 0:
            relevancia_norm = funcao_ativacao_relevancia_v2(res['relevancia_contextual'])
            score_norm = normalizar(res['total_score'], kwargs_dinamicos['min_score'], kwargs_dinamicos['max_score'])
            peso_medio_norm = normalizar(res['avg_word_weight'], kwargs_dinamicos['min_peso'], kwargs_dinamicos['max_peso'])
            tema_norm = normalizar(res['score_tematico'], kwargs_dinamicos['min_tema'], kwargs_dinamicos['max_tema'])

            dados_grafico.append({'Site': site, 'Componente': 'Tema', 'Contribuição': (tema_norm * PESO_TEMA)})
            dados_grafico.append({'Site': site, 'Componente': 'Relevância', 'Contribuição': (relevancia_norm * PESO_RELEVANCIA)})
            dados_grafico.append({'Site': site, 'Componente': 'Score Bruto', 'Contribuição': (score_norm * PESO_SCORE)})
            dados_grafico.append({'Site': site, 'Componente': 'Peso Médio', 'Contribuição': (peso_medio_norm * PESO_MEDIO)})

    return pd.DataFrame(dados_grafico)

# --- Preparação dos DataFrames para os Gráficos ---

# 1. DataFrame para o gráfico de barras empilhadas
df_scores_componentes = obter_dados_para_dashboard_scores(resultados_finais)

# 2. DataFrame para o ranking geral (também com nomes curtos)
df_ranking = pd.DataFrame(resultados_finais)
df_ranking['Site'] = df_ranking['site_url'].apply(encurtar_url)
# Filtramos os sites com nota 0 para um gráfico mais limpo
df_ranking = df_ranking[df_ranking['nota_final'] > 0].sort_values('nota_final', ascending=False)

print("Dados para os dashboards foram preparados com sucesso!")

Dados para os dashboards foram preparados com sucesso!


In [32]:
# Inicializa a aplicação Dash
app_sites = dash.Dash(__name__)

# Layout do Dashboard
app_sites.layout = html.Div(style={'backgroundColor': '#f9f9f9', 'fontFamily': 'Verdana'}, children=[
    html.H1(
        "Dashboard de Análise e Ranking de Sites",
        style={'textAlign': 'center', 'color': '#2c3e50', 'padding': '20px'}
    ),

    # Gráfico de Ranking Geral
    html.Div([
        html.H2("Ranking Final dos Sites", style={'textAlign': 'center', 'color': '#34495e'}),
        dcc.Graph(
            id='grafico-ranking-sites',
            figure=px.bar(
                df_ranking,
                x='nota_final',
                y='Site',
                orientation='h', # Gráfico de barras horizontais
                title='Nota Final dos Sites (de 0 a 10)',
                labels={'Site': '', 'nota_final': 'Nota Final'},
                text='nota_final',
                template='plotly_white'
            ).update_traces(
                texttemplate='%{text:.2f}', textposition='outside', marker_color='#27ae60'
            ).update_yaxes(categoryorder="total ascending") # Ordena do menor para o maior
             .update_layout(title_x=0.5, yaxis_title=None, xaxis_title='Nota Final')
        )
    ], style={'padding': '20px', 'border': '1px solid #ddd', 'borderRadius': '5px', 'margin': '10px'}),

    html.Hr(),

    # Gráfico de Barras Empilhadas
    html.Div([
        html.H2("Detalhamento da Pontuação por Componente", style={'textAlign': 'center', 'color': '#34495e'}),
        dcc.Graph(
            id='grafico-componentes-score',
            figure=px.bar(
                df_scores_componentes,
                x='Contribuição',
                y='Site',
                orientation='h',
                color='Componente',
                title='Composição da Nota Final (Componentes Normalizados)',
                labels={'Site': '', 'Contribuição': 'Valor da Contribuição para a Nota'},
                barmode='stack', # Barras empilhadas
                color_discrete_map={
                    'Tema': '#3498db',
                    'Relevância': '#f1c40f',
                    'Score Bruto': '#2ecc71',
                    'Peso Médio': '#e74c3c'
                }
            ).update_layout(
                title_x=0.5,
                template='gridon',
                legend_title_text='Componentes da Nota',
                yaxis_title=None,
                yaxis=dict(categoryorder='total ascending') # Garante a mesma ordem do gráfico de ranking
            )
        )
    ], style={'padding': '20px', 'border': '1px solid #ddd', 'borderRadius': '5px', 'margin': '10px'})
])

# Para rodar no Google Colab/Jupyter
if __name__ == '__main__':
    app_sites.run(mode='inline', height=1000)

<IPython.core.display.Javascript object>